In [43]:
import warnings
import pymysql
import re
from pyhanlp import *
import json
warnings.filterwarnings('ignore')
CRFnewSegment = HanLP.newSegment("crf")
HanLP.Config.ShowTermNature = True
with open('停用词表','r',encoding='utf-8') as f:
    stop_words = f.read()
stop_words = stop_words.split('\n')
stop_words = stop_words + ['表示','说','转发','转','分享','快转','微博','快转微博','发','转发微博','分享图片','分享','图片']
Part_of_speechs = ['b','c','d','dg','e','f','g','o','p','q','r','u','ud','ug','uj','ul','uv','uz','w','x','y','z']
#区别词,连词,副词,副语素,叹词,方位词,语素,拟声词,量词,代词
conn = pymysql.connect(port=3306,user='root',passwd='220155',charset='utf8')
cursor = conn.cursor()
sql = 'use weibo_media'
cursor.execute(sql)
sql = "select post_content from Post"
cursor.execute(sql)
conn.commit()
documents = cursor.fetchall()


In [75]:
documents_clean = []
for c, sentence in enumerate(documents):
    sentence = sentence[-1]
    # 提取话题
    # topics = re.findall("<#>.*?</#>",data)
    topics = re.findall("(?<=<#>).*?(?=</#>)", sentence)
    data = re.sub("<#>.*?</#>", '', sentence)
    # print('话题:',topics)
    # 提取链接
    # links = re.findall("<u>.*?</u>", data)
    links = re.findall("(?<=<u>).*?(?=</u>)", sentence)
    data = re.sub("<u>.*?</u>", '', sentence)
    # print('链接:', links)
    # 提取用户
    # users = re.findall("<@>.*?</@>", data)
    users = re.findall("(?<=<@>).*?(?=</@>)", sentence)
    data = re.sub("<@>.*?</@>", '', data)
    # 去除英文
    sentence = re.sub("[^\u4e00-\u9fa5]", '', sentence)
    document = []
    # 分词识别词性
    words = CRFnewSegment.seg(sentence)
    for term in words:
        word = str(term.word)
        nature = str(term.nature)
        if nature in Part_of_speechs or word in stop_words:
            continue
        document.append(word)
    documents_clean.append(document)

In [76]:
from gensim.models import LdaModel
from gensim.corpora import Dictionary

In [77]:
dictionary = Dictionary(documents_clean)
documents_matrix = [dictionary.doc2bow(doc) for doc in documents_clean]

In [78]:
lda = LdaModel(documents_matrix,num_topics=20,id2word=dictionary)

In [90]:
lda.show_topics(num_topics=20)

[(0,
  '0.043*"伊朗" + 0.028*"美国" + 0.019*"特朗普" + 0.012*"袭击" + 0.011*"伊拉克" + 0.008*"美军" + 0.007*"称" + 0.006*"讲话" + 0.006*"导弹" + 0.006*"视频"'),
 (1,
  '0.073*"伊朗" + 0.051*"美国" + 0.017*"袭击" + 0.014*"伊拉克" + 0.012*"称" + 0.012*"特朗普" + 0.012*"美军" + 0.010*"苏莱曼尼" + 0.009*"视频" + 0.007*"总统"'),
 (2,
  '0.046*"伊朗" + 0.029*"美国" + 0.020*"特朗普" + 0.013*"称" + 0.009*"伊拉克" + 0.008*"报道" + 0.008*"美军" + 0.007*"空袭" + 0.006*"苏莱曼尼" + 0.005*"总统"'),
 (3,
  '0.047*"美国" + 0.044*"伊朗" + 0.012*"伊拉克" + 0.012*"袭击" + 0.011*"特朗普" + 0.010*"中东" + 0.008*"威胁" + 0.007*"目标" + 0.007*"称" + 0.007*"总统"'),
 (4,
  '0.067*"伊朗" + 0.049*"美国" + 0.012*"伊拉克" + 0.010*"制裁" + 0.010*"袭击" + 0.007*"称" + 0.007*"核" + 0.007*"报复" + 0.006*"时间" + 0.006*"协议"'),
 (5,
  '0.066*"伊朗" + 0.028*"美国" + 0.024*"协议" + 0.021*"伊" + 0.020*"核" + 0.013*"称" + 0.011*"特朗普" + 0.008*"伊拉克" + 0.008*"美军" + 0.008*"袭击"'),
 (6,
  '0.055*"美国" + 0.048*"伊朗" + 0.020*"特朗普" + 0.014*"中东" + 0.012*"总统" + 0.011*"伊拉克" + 0.010*"袭击" + 0.008*"美军" + 0.007*"发表" + 0.007*"称"'),
 (7,
  '0.044*"伊朗" +

In [79]:
clusters = {}
for i,doc in enumerate(documents_matrix):
    category = lda[doc][0][0]
    if category in clusters:
        clusters[category].append(i)
    else:
        clusters[category] = [i]

In [83]:
for i,v in clusters.items():
    print(len(v))

124
282
108
48
111
40
51
117
92
164
64
125
84
73
19
37
9
44
61
12


In [80]:
for i,v in clusters.items():
    print(i)
    for j in v:
        print(documents[j][-1])
    print("<->"*100)

15
【伊朗特种部队指挥官伊拉克遭火箭弹袭击身亡，外媒：美国声称在巴格达发动“与伊朗有关的”袭击】当地时间周五，伊拉克巴格达附近遭不明火箭弹袭击，导致伊拉克什叶派民兵武装“人民动员组织”5名成员死亡，包括该组织副指挥官，此外，还有两名该组织重要“伊朗客人”死亡，据中东媒体等报道，两名客人中包括伊朗伊斯兰革命卫队下属特种部队“圣城旅”旅长卡西姆∙苏莱曼尼。目前未有组织和个人宣布对此起袭击负责，但英媒援引消息人士的话报道称，美军当天空袭了巴格达两个与伊朗有联系的目标。最新消息，据路透社报道，美国证实，疑与伊朗有关的巴格达机场袭击系美国所为。当地时间3日凌晨，三枚火箭弹落在伊拉克首都巴格达国际机场附近，爆炸造成至少8人死亡，12名伊拉克军人受伤。此前，美国驻伊拉克大使馆遭示威者冲击破坏，美方指责伊朗系“幕后黑手”。<#>#伊朗特种部队司令伊拉克身亡#</#><#>#伊朗少将在伊拉克遇袭身亡#</#><#>#伊拉克再遭美军空袭#</#>（via.微天下）
【#伊朗拟悬赏8000万刺杀特朗普#？谣言！】据伊朗国家电视台称，伊朗准备悬赏8000万美元刺杀特朗普。“伊朗有8000万居民，将募集8000万美元，奖励刺杀特朗普的人，每个伊朗人出资1美元来支付这笔悬赏金”。后，美国知名事实核查网站“Snopes”称，“悬赏的呼吁尚未得到伊朗官方授权，不代表其官方立场。”#伊朗悬赏8000万美元刺杀特朗普#
【<#>#特朗普就伊朗袭击发表讲话#</#>：无美方人员在袭击中死亡；永不允许伊朗拥有核武器】当地时间1月8日，#特朗普#对美驻伊拉克军事基地遭袭一事发表全国讲话。特朗普说：“只要我还是美国总统，就永远不允许伊朗拥有核武器。”，<#>#特朗普称无美军在袭击中死亡#</#>，他还将苏莱曼尼形容为“世界头号恐怖分子”。并称“是我们阻止了他袭击美国目标”。<u>L时差视频的微博视频:['http://t.cn/Ais9QoTI']</u>
【<#>#美众议院通过限制总统对伊朗动武决议#</#>】美国国会众议院9日通过一项旨在限制总统对伊朗动武权力的决议。（新华社）
【<#>#美驻伊使馆附近及空军基地遭袭#</#>，<#>#特朗普警告已锁定伊朗52个攻击目标#</#>】据BBC报道，1月4日，大量民众参与伊朗高级将领苏莱曼尼的葬礼游行。随后，几枚火箭弹袭击了巴格达市中心设有美国使馆的绿区和北

In [22]:
from gensim.models import LdaModel
from gensim.corpora import Dictionary
doc_clean = [doc.split() for doc in doc_complete]

In [23]:
# 创建语料的词语词典，每个单独的词语都会被赋予一个索引
dictionary = Dictionary(doc_clean)#id2word
# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵，文档转换成词袋模型
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [26]:
# 在 DT 矩阵上运行和训练 LDA 模型
ldamodel = LdaModel(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

In [81]:
# 输出结果
print(ldamodel.print_topics(num_topics=3, num_words=3))

[(0, '0.093*"to" + 0.064*"my" + 0.049*"sister"'), (1, '0.046*"driving" + 0.046*"that" + 0.046*"and"'), (2, '0.046*"Sugar" + 0.046*"is" + 0.046*"not"')]


In [28]:
ldamodel.num_topics

3

In [31]:
doc_bow = dictionary.doc2bow(doc_clean[0])
ldamodel[doc_bow] 

[(0, 0.9560786), (1, 0.02107762), (2, 0.022843778)]